In [ ]:
from transformers import pipeline
import pandas as pd
import torch

# CSV dosyasını yükle
file_path = "All_Scrapping_Cleaned_Tokenized_StopWord_lemmatized.csv"
data = pd.read_csv(file_path)

# Geçerli (dolu) yorumları filtrele
yorumlar = data['Yorum Temiz'].dropna().tolist()

# Token sınırını kontrol etmek ve gerekirse kırpmak için bir yardımcı fonksiyon
def truncate_text(text, max_length=512):
    return text[:max_length]

yorumlar = [truncate_text(yorum) for yorum in yorumlar]

# GPU kontrolü ve modeli yükle
device = 0 if torch.cuda.is_available() else -1  # GPU kullanılabilir ise 0, değilse -1
print(f"Model şu cihazda çalışacak: {'GPU' if device == 0 else 'CPU'}")

# Türkçe duygu analizi için modeli yükle
sentiment_analyzer = pipeline("sentiment-analysis", model="savasy/bert-base-turkish-sentiment-cased", device=device) #Türçe duygu analizi yapmak için ekledik

# Duygu analizi yap
etiketler = sentiment_analyzer(yorumlar)#yorumları kullanarak duygu analizi yapıldı

# Analiz edilen yorumları ve etiketlerini birleştir. Etiketler 1 veya 0 olarak gerçekleştirildi.
data_filtered = data.dropna(subset=['Yorum Temiz']).reset_index(drop=True)
data_filtered['Duygu Etiketi'] = [result['label'] for result in etiketler]

# Sonuçları yeni bir csv dosyasına kaydetti.
output_path = "All_Scrapping_with_Sentiment_Labels_Truncated2.csv"
data_filtered.to_csv(output_path, index=False)
print(f"Duygu analizi sonuçları kaydedildi: {output_path}")
